In [167]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv("total_combined_normal.csv")
df.drop(['Unnamed: 0'],axis=1, inplace = True)
print(df.shape)

# df = pd.concat([df, pd.get_dummies(df.T_ID)], axis = 1)
# data.drop(["T_ID"], axis = 1, inplace = True)

(3200, 76)


# 1. 승무패 불러오기

In [255]:
year = [2016,2017,2018,2019,2020]
team = []
team_pitcher = []

for i in year:
    team_pitcher.append(pd.read_excel('excel_data/2020빅콘테스트_스포츠투아이_제공데이터_{}.xlsx'.format(i), sheet_name = '4.팀투수'))
team_pitcher_df = pd.concat(team_pitcher, axis=0)

In [91]:
wls = team_pitcher_df[["G_ID","T_ID","VS_T_ID","WLS"]]
wls = wls.rename({'WLS':'wls'},axis=1)
wls

,G_ID,T_ID,VS_T_ID,wls
0,20160401HHLG0,LG,HH,W
1,20160401HHLG0,HH,LG,L
2,20160401HTNC0,NC,HT,W
3,20160401HTNC0,HT,NC,L
4,20160401KTSK0,SK,KT,L
...,...,...,...,...
635,20200719LTSS0,LT,SS,W
636,20200719OBHT0,HT,OB,L
637,20200719OBHT0,OB,HT,W
638,20200719WOSK0,SK,WO,W


# 2. 경기 분리

In [100]:
reverse_df = df.copy()

col = list(df.columns)
re_col = {'T_ID': 'VS_T_ID', 'VS_T_ID':'T_ID'}
for i in range(6,len(col)):
    if 'vs' in col[i]:
        re_col[col[i]] = col[i][3:]    
    else:
        re_col[col[i]] = 'vs_'+col[i]
        
reverse_df= reverse_df.rename(re_col,axis=1)

tb_sc_dict = {'T':'B', 'B':'T'}
for i in range(len(reverse_df)):
    orig = df.loc[i,"TB_SC"]
    reverse_df.loc[i,"TB_SC"] = tb_sc_dict[orig]
    

In [104]:
data = pd.concat([df, reverse_df],axis=0)
data = data.sort_values(by = ["G_ID"]).reset_index(drop = True)

# 3. 승무패 붙이기

In [106]:
fin_data = pd.merge(data, wls, how='left')

wls_dict = {'W':'L', 'D':'D', 'L':'W'}
for i in range(len(fin_data)):
    now_wls = fin_data.loc[i,'wls']
    fin_data.loc[i,'vs_wls'] = wls_dict[now_wls]

# 4. 연도 추가/상대팀 one-hot 추가

In [151]:
# 연도추가
fin_data.loc[:,'YEAR'] = fin_data.loc[:,'G_ID'].str.slice(start= 0, stop=4)

# 상대팀 one-hot
team = list(df.T_ID.unique())
team_dict_vstid = dict()
for i in team:
    team_dict_vstid[i] = 'vs_'+i

add_data = pd.concat([fin_data, pd.get_dummies(fin_data.VS_T_ID)], axis = 1)
add_data = add_data.rename(team_dict_vstid, axis =1)

In [152]:
add_data

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,WHIP,LOB,ERA,FIP,...,vs_HH,vs_HT,vs_KT,vs_LG,vs_LT,vs_NC,vs_OB,vs_SK,vs_SS,vs_WO
0,20160401HHLG0,20160401,LG,HH,0,T,1.333,0.750,3.000,7.333,...,1,0,0,0,0,0,0,0,0,0
1,20160401HHLG0,20160401,HH,LG,0,B,1.147,0.690,3.176,23.059,...,0,0,0,1,0,0,0,0,0,0
2,20160401HTNC0,20160401,NC,HT,0,T,1.222,0.755,4.000,22.778,...,0,1,0,0,0,0,0,0,0,0
3,20160401HTNC0,20160401,HT,NC,0,B,1.750,0.804,5.625,38.750,...,0,0,0,0,0,1,0,0,0,0
4,20160401KTSK0,20160401,KT,SK,0,B,1.111,0.698,4.000,11.444,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,20200719,LT,SS,0,B,1.111,1.047,1.000,20.000,...,0,0,0,0,0,0,0,0,1,0
6396,20200719OBHT0,20200719,OB,HT,0,B,1.556,0.714,4.000,16.889,...,0,1,0,0,0,0,0,0,0,0
6397,20200719OBHT0,20200719,HT,OB,0,T,1.444,0.515,8.000,21.111,...,0,0,0,0,0,0,1,0,0,0
6398,20200719WOSK0,20200719,SK,WO,0,T,1.111,0.833,3.000,24.111,...,0,0,0,0,0,0,0,0,0,1


In [153]:
# add_data.to_csv("preprocessing_csv/original_data.csv", index = False)

# 5. 연도-팀별 24경기씩 통계
#### (타율/방어율/승률:sum, 나머지:mean) 

<전체 합에 대해>
- 승률: 이긴횟수/(이긴횟수+진횟수)
- 타율: 안타(HIT)/타수(AB)
- 평균자책: 자책점(ER) X 9 / 이닝(INN2/3)

In [181]:
add_col = list(add_data.columns)

default_col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID',"YEAR"]
size_col = ['wls','vs_wls']
sum_col = ['HEADER_NO',
           'PITCHER_INNING', 'PITCHER_ER', 'HITTER_AB','HITTER_HIT',  
           'vs_PITCHER_INNING', 'vs_PITCHER_ER','vs_HITTER_AB', 'vs_HITTER_HIT', 
           'vs_HH', 'vs_HT','vs_KT', 'vs_LG', 'vs_LT', 'vs_NC', 'vs_OB', 'vs_SK', 'vs_SS', 'vs_WO']

vsteam_col = ['vs_HH', 'vs_HT','vs_KT', 'vs_LG', 'vs_LT', 'vs_NC', 'vs_OB', 'vs_SK', 'vs_SS', 'vs_WO']

tmp = set(default_col)|set(size_col)|set(sum_col)
mean_col = list(set(add_col)-tmp)

In [155]:
size_df = add_data[default_col + size_col]
sum_df = add_data[default_col + sum_col]
mean_df = add_data[default_col + mean_col]

#### size

In [172]:
def make_pct(df):
    tmp = pd.DataFrame({"COUNT":df.groupby(["T_ID","wls","YEAR"]).size()}).pivot_table("COUNT",["T_ID","YEAR"],"wls").reset_index().sort_values(["YEAR","T_ID"]).reset_index(drop=True)
    tmp.fillna(0,inplace= True)

    if "L" not in list(tmp.columns):
        tmp["L"] = 0

    if "W" not in list(tmp.columns):
        tmp["W"] = 0
    
    tmp["PCT"] = tmp['W']/(tmp['W']+tmp['L'])
    return tmp

In [221]:
size_df_cal = pd.DataFrame([])

idx = 0
key = 24
for y in [2016,2017,2018,2019]:
    print("===",y)
    
    y_tmp = size_df[size_df["YEAR"]==str(y)]
    for t in team:
        print(t)
        tmp_df = y_tmp[y_tmp["T_ID"]==t]
        
        for k in range(6):
            tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
            tmp2 = make_pct(tmp1)
            tmp2['IDX'] = k+1

            size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)
            idx += 1
            
size_df_cal.drop(["W","D","L"],axis = 1, inplace = True)
        
# df16 = pd.DataFrame([])

# idx = 0
# for t in list(team["T_ID"]):
#     print(t)
#     df = tmp16_pre[tmp16_pre["T_ID"]==t]
#     for k in range(12):
#         tmp1 = df.iloc[12*k:12*(k+1),:]
#         tmp2 = make_pct(tmp1)
#         tmp2['IDX'] = k+1
        
#         df16 = pd.concat([df16, tmp2], axis = 0)
#         idx += 1

=== 2016
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2017
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2018
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2019
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT


In [222]:
size_df_cal.head(6)

,T_ID,YEAR,PCT,IDX
0,LG,2016,0.500000,1
0,LG,2016,0.478261,2
0,LG,2016,0.375000,3
0,LG,2016,0.458333,4
0,LG,2016,0.583333,5
0,LG,2016,0.608696,6


#### sum

<전체 합에 대해>
- 승률: 이긴횟수/(이긴횟수+진횟수)
- 타율: 안타(HIT)/타수(AB)
- 평균자책: 자책점(ER) X 9 / 이닝(INN2/3)

In [245]:
def make_sum(df):
    tmp = df.groupby(["T_ID","YEAR"]).sum().reset_index()
    
    # ERA
    tmp["era"] = tmp["PITCHER_ER"]*9/tmp["PITCHER_INNING"]
    tmp["vs_era"] = tmp["vs_PITCHER_ER"]*9/tmp["vs_PITCHER_INNING"]
    
    #AVG
    tmp['avg'] = tmp["HITTER_HIT"]/tmp["HITTER_AB"]
    tmp['vs_avg'] = tmp["vs_HITTER_HIT"]/tmp["vs_HITTER_AB"]
    
    re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"] + vsteam_col
    return tmp[re_col]
    

In [246]:
sum_df_cal = pd.DataFrame([])

idx = 0
key = 24
for y in [2016,2017,2018,2019]:
    print("===",y)
    
    y_tmp = sum_df[sum_df["YEAR"]==str(y)]
    for t in team:
        print(t)
        tmp_df = y_tmp[y_tmp["T_ID"]==t]
        
        for k in range(6):
            tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
            tmp2 = make_sum(tmp1)
            tmp2['IDX'] = k+1

            sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
            idx += 1

=== 2016
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2017
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2018
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2019
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT


In [247]:
sum_df_cal.head(6)

,T_ID,YEAR,HEADER_NO,era,avg,vs_era,vs_avg,vs_HH,vs_HT,vs_KT,vs_LG,vs_LT,vs_NC,vs_OB,vs_SK,vs_SS,vs_WO,IDX
0,LG,2016,0,5.358453,0.253317,4.555222,0.298349,4,2,3,0,3,3,1,3,2,3,1
0,LG,2016,0,4.715603,0.291169,4.659483,0.274605,0,3,4,0,2,3,4,2,3,3,2
0,LG,2016,0,5.584768,0.305842,5.148837,0.288889,3,6,2,0,0,2,0,5,3,3,3
0,LG,2016,0,5.551402,0.297110,5.833333,0.288942,3,0,1,0,6,3,6,0,2,3,4
0,LG,2016,0,5.400000,0.294471,5.091926,0.286756,4,2,4,0,2,2,2,4,2,2,5
0,LG,2016,0,3.649770,0.296069,4.993723,0.268530,2,3,2,0,3,3,3,2,4,2,6


#### mean

In [248]:
def make_mean(df):
    tmp = df.groupby(["T_ID","YEAR"]).mean().reset_index()
    tmp.drop(["GDAY_DS"],axis=1,inplace = True)
    return tmp

In [249]:
mean_df_cal = pd.DataFrame([])

idx = 0
key = 24
for y in [2016,2017,2018,2019]:
    print("===",y)
    
    y_tmp = mean_df[mean_df["YEAR"]==str(y)]
    for t in team:
        print(t)
        tmp_df = y_tmp[y_tmp["T_ID"]==t]
        
        for k in range(6):
            tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
            tmp2 = make_mean(tmp1)
            tmp2['IDX'] = k+1

            mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
            idx += 1

=== 2016
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2017
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2018
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2019
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT


In [250]:
mean_df_cal.head(12)

,T_ID,YEAR,vs_XR,OPS,vs_ERA,vs_H_A,ERA,vs_HR_9,OBP,vs_P_WHIP_RT,...,P_HRA_RT,PF,vs_GPA,vs_BA,XR,vs_RC,vs_WHIP,FIP,vs_oAVG,IDX
0,LG,2016,5.260250,0.699750,4.564292,0.500000,5.445125,1.132250,0.325583,1.419500,...,0.216875,972.833333,0.266833,0.289208,4.715583,0.870292,1.417625,20.730708,0.249250,1
0,LG,2016,4.609000,0.758833,4.835125,0.500000,4.840875,0.589875,0.355625,1.796958,...,0.279667,973.000000,0.249500,0.268792,4.999083,0.845500,1.526958,18.914375,0.288875,2
0,LG,2016,5.693083,0.774292,5.223208,0.458333,5.642250,0.725458,0.363542,1.620750,...,0.249625,968.625000,0.272500,0.285500,5.559417,0.994750,1.615917,27.979500,0.307333,3
0,LG,2016,5.810750,0.798417,5.848958,0.583333,5.592792,0.917625,0.366333,1.930458,...,0.290625,981.000000,0.274167,0.280625,5.866500,1.168750,1.644917,27.325833,0.296250,4
0,LG,2016,5.075083,0.761125,5.106250,0.583333,5.534375,0.807500,0.346875,1.341375,...,0.261875,977.666667,0.261167,0.280750,4.921000,0.981042,1.514083,21.093500,0.295083,5
0,LG,2016,4.364250,0.777250,5.183583,0.375000,3.755667,0.779958,0.364917,1.577083,...,0.280875,974.541667,0.245708,0.265625,5.104000,0.508167,1.588458,19.183417,0.289250,6
0,NC,2016,3.750833,0.737292,4.561083,0.625000,3.717333,1.018458,0.337375,1.640375,...,0.246792,1007.125000,0.224375,0.232958,4.601750,0.246583,1.425250,18.615250,0.263833,1
0,NC,2016,4.895167,0.874292,6.870833,0.416667,4.535417,1.270833,0.397667,2.252542,...,0.321125,1024.041667,0.253875,0.263958,6.688417,0.658208,1.834458,25.014750,0.309875,2
0,NC,2016,5.240500,0.829292,6.697917,0.583333,5.135417,1.598958,0.365583,2.108917,...,0.303625,1003.958333,0.263292,0.281042,6.346083,0.853625,1.709500,23.039458,0.284417,3
0,NC,2016,4.943750,0.839625,5.589750,0.500000,5.177083,1.513625,0.369625,1.751458,...,0.292708,1025.916667,0.258042,0.273667,5.912917,0.770250,1.601000,22.495042,0.285417,4


# 6. 3개의 table 합치기

In [251]:
tmp_table = pd.merge(size_df_cal, sum_df_cal, how= 'left', left_on = ['T_ID',"YEAR","IDX"], right_on  = ['T_ID',"YEAR","IDX"])
merge_df = pd.merge(tmp_table, mean_df_cal, how = 'left', left_on = ['T_ID',"YEAR","IDX"], right_on  = ['T_ID',"YEAR","IDX"] )

In [254]:
# merge_df.to_csv("preprocessing_csv/lstm_24group_1619.csv", index = False)